In [1]:
import sqlalchemy
import pandas as pd
import numpy as np
import sql_functions as sf

from sql_functions import *
import psycopg2

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns

In [2]:
schema = 'capstone_jmrs'
spotahome_eda = sf.get_dataframe(f'SELECT * FROM {schema}.spotahome_merged')
spotahome_eda

,platform_id,platform,neighborhood,property_type,housing_type,price_pcm,title,furniture,price_pcm_0,price_pcm_1,bathrooms,size_sqm,bedrooms,available_from
0,654316,spotahome,Kensington and Chelsea,apartments/bedrooms:1,Apartment,3690,Stunning Chelsea Nest,furnished,3690,NaN,1.0,NaN,1.0,2023-04-02
1,726546,spotahome,Lambeth,apartments/bedrooms:2,Apartment,3924-8856,2-bedroom apartment for rent in Patmore Estate...,furnished,3924,8856.0,2.0,72.0,2.0,2022-11-06
2,438447,spotahome,Kensington and Chelsea,studios,Studio,3040,Studio in London,furnished,3040,NaN,1.0,21.0,0.0,2023-01-01
3,382488,spotahome,Camden,studios,Studio,1755,"Studio flat to rent in Camden Town, London",furnished,1755,NaN,1.0,NaN,0.0,2023-01-11
4,619142,spotahome,Lambeth,apartments/bedrooms:1,Apartment,4500,"One bedroom apartment for rent in Brixton, London",furnished,4500,NaN,1.0,NaN,1.0,2022-10-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831,643947,spotahome,Islington,apartments/bedrooms:3,Apartment,7350,"3-bedroom apartment for rent in Islington, London",furnished,7350,NaN,3.0,NaN,3.0,2022-11-07
832,641245,spotahome,Hackney,apartments/bedrooms:3,Apartment,6000,"3-bedroom apartment for rent in Haggerston, Lo...",furnished,6000,NaN,NaN,NaN,3.0,2022-11-26
833,506930,spotahome,Hackney,apartments/bedrooms:3,Apartment,4941,"3-Bedroom Apartment for rent in Old Street, Lo...",furnished,4941,NaN,2.0,90.0,3.0,2023-01-14
834,843222,spotahome,Hackney,apartments/bedrooms:3,Apartment,8250-10500,3-bedroom apartment for rent in London,furnished,8250,10500.0,1.0,NaN,3.0,2023-01-08


In [3]:
spotahome_eda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 836 entries, 0 to 835
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   platform_id     836 non-null    int64  
 1   platform        836 non-null    object 
 2   neighborhood    836 non-null    object 
 3   property_type   836 non-null    object 
 4   housing_type    836 non-null    object 
 5   price_pcm       836 non-null    object 
 6   title           836 non-null    object 
 7   furniture       836 non-null    object 
 8   price_pcm_0     836 non-null    int64  
 9   price_pcm_1     247 non-null    float64
 10  bathrooms       519 non-null    float64
 11  size_sqm        262 non-null    float64
 12  bedrooms        835 non-null    float64
 13  available_from  836 non-null    object 
dtypes: float64(4), int64(2), object(8)
memory usage: 91.6+ KB


In [4]:
spotahome_eda.describe()

,platform_id,price_pcm_0,price_pcm_1,bathrooms,size_sqm,bedrooms
count,836.000000,836.000000,247.000000,519.000000,262.000000,835.000000
mean,547292.527512,4399.885167,7997.336032,1.315992,60.110687,1.259880
std,159792.599859,1938.790440,3801.954404,0.636983,35.074075,1.100799
min,98665.000000,1112.000000,2881.000000,1.000000,12.000000,0.000000
25%,422259.250000,3090.000000,5500.000000,1.000000,40.000000,0.000000
50%,573947.500000,4125.000000,7200.000000,1.000000,55.000000,1.000000
75%,642348.250000,5221.250000,9218.500000,1.000000,75.000000,2.000000
max,892606.000000,13614.000000,27372.000000,5.000000,350.000000,6.000000


In [5]:
spotahome_eda.corr()

,platform_id,price_pcm_0,price_pcm_1,bathrooms,size_sqm,bedrooms
platform_id,1.000000,0.174076,0.040142,0.144353,0.156237,-0.007917
price_pcm_0,0.174076,1.000000,0.775776,0.489896,0.410722,0.521323
price_pcm_1,0.040142,0.775776,1.000000,0.653239,0.688852,0.720156
bathrooms,0.144353,0.489896,0.653239,1.000000,0.731517,0.620871
size_sqm,0.156237,0.410722,0.688852,0.731517,1.000000,0.802987
bedrooms,-0.007917,0.521323,0.720156,0.620871,0.802987,1.000000


In [9]:
spotahome_eda['avg_price_sqm'] = spotahome_eda[['price_pcm_0', 'price_pcm_1']].mean(axis=1, skipna=True)

In [10]:
spotahome_eda

,platform_id,platform,neighborhood,property_type,housing_type,price_pcm,title,furniture,price_pcm_0,price_pcm_1,bathrooms,size_sqm,bedrooms,available_from,avg_price_sqm
0,654316,spotahome,Kensington and Chelsea,apartments/bedrooms:1,Apartment,3690,Stunning Chelsea Nest,furnished,3690,NaN,1.0,NaN,1.0,2023-04-02,3690.0
1,726546,spotahome,Lambeth,apartments/bedrooms:2,Apartment,3924-8856,2-bedroom apartment for rent in Patmore Estate...,furnished,3924,8856.0,2.0,72.0,2.0,2022-11-06,6390.0
2,438447,spotahome,Kensington and Chelsea,studios,Studio,3040,Studio in London,furnished,3040,NaN,1.0,21.0,0.0,2023-01-01,3040.0
3,382488,spotahome,Camden,studios,Studio,1755,"Studio flat to rent in Camden Town, London",furnished,1755,NaN,1.0,NaN,0.0,2023-01-11,1755.0
4,619142,spotahome,Lambeth,apartments/bedrooms:1,Apartment,4500,"One bedroom apartment for rent in Brixton, London",furnished,4500,NaN,1.0,NaN,1.0,2022-10-04,4500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831,643947,spotahome,Islington,apartments/bedrooms:3,Apartment,7350,"3-bedroom apartment for rent in Islington, London",furnished,7350,NaN,3.0,NaN,3.0,2022-11-07,7350.0
832,641245,spotahome,Hackney,apartments/bedrooms:3,Apartment,6000,"3-bedroom apartment for rent in Haggerston, Lo...",furnished,6000,NaN,NaN,NaN,3.0,2022-11-26,6000.0
833,506930,spotahome,Hackney,apartments/bedrooms:3,Apartment,4941,"3-Bedroom Apartment for rent in Old Street, Lo...",furnished,4941,NaN,2.0,90.0,3.0,2023-01-14,4941.0
834,843222,spotahome,Hackney,apartments/bedrooms:3,Apartment,8250-10500,3-bedroom apartment for rent in London,furnished,8250,10500.0,1.0,NaN,3.0,2023-01-08,9375.0


In [13]:
spotahome_eda.eval('price_per_sqm = avg_price_sqm / size_sqm', inplace=True)

In [14]:
spotahome_eda

,platform_id,platform,neighborhood,property_type,housing_type,price_pcm,title,furniture,price_pcm_0,price_pcm_1,bathrooms,size_sqm,bedrooms,available_from,avg_price_sqm,price_per_sqm
0,654316,spotahome,Kensington and Chelsea,apartments/bedrooms:1,Apartment,3690,Stunning Chelsea Nest,furnished,3690,NaN,1.0,NaN,1.0,2023-04-02,3690.0,NaN
1,726546,spotahome,Lambeth,apartments/bedrooms:2,Apartment,3924-8856,2-bedroom apartment for rent in Patmore Estate...,furnished,3924,8856.0,2.0,72.0,2.0,2022-11-06,6390.0,88.750000
2,438447,spotahome,Kensington and Chelsea,studios,Studio,3040,Studio in London,furnished,3040,NaN,1.0,21.0,0.0,2023-01-01,3040.0,144.761905
3,382488,spotahome,Camden,studios,Studio,1755,"Studio flat to rent in Camden Town, London",furnished,1755,NaN,1.0,NaN,0.0,2023-01-11,1755.0,NaN
4,619142,spotahome,Lambeth,apartments/bedrooms:1,Apartment,4500,"One bedroom apartment for rent in Brixton, London",furnished,4500,NaN,1.0,NaN,1.0,2022-10-04,4500.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831,643947,spotahome,Islington,apartments/bedrooms:3,Apartment,7350,"3-bedroom apartment for rent in Islington, London",furnished,7350,NaN,3.0,NaN,3.0,2022-11-07,7350.0,NaN
832,641245,spotahome,Hackney,apartments/bedrooms:3,Apartment,6000,"3-bedroom apartment for rent in Haggerston, Lo...",furnished,6000,NaN,NaN,NaN,3.0,2022-11-26,6000.0,NaN
833,506930,spotahome,Hackney,apartments/bedrooms:3,Apartment,4941,"3-Bedroom Apartment for rent in Old Street, Lo...",furnished,4941,NaN,2.0,90.0,3.0,2023-01-14,4941.0,54.900000
834,843222,spotahome,Hackney,apartments/bedrooms:3,Apartment,8250-10500,3-bedroom apartment for rent in London,furnished,8250,10500.0,1.0,NaN,3.0,2023-01-08,9375.0,NaN


In [15]:
spotahome_eda['price_per_sqm'] = spotahome_eda['price_per_sqm'].round(decimals = 2)

In [16]:
spotahome_eda

,platform_id,platform,neighborhood,property_type,housing_type,price_pcm,title,furniture,price_pcm_0,price_pcm_1,bathrooms,size_sqm,bedrooms,available_from,avg_price_sqm,price_per_sqm
0,654316,spotahome,Kensington and Chelsea,apartments/bedrooms:1,Apartment,3690,Stunning Chelsea Nest,furnished,3690,NaN,1.0,NaN,1.0,2023-04-02,3690.0,NaN
1,726546,spotahome,Lambeth,apartments/bedrooms:2,Apartment,3924-8856,2-bedroom apartment for rent in Patmore Estate...,furnished,3924,8856.0,2.0,72.0,2.0,2022-11-06,6390.0,88.75
2,438447,spotahome,Kensington and Chelsea,studios,Studio,3040,Studio in London,furnished,3040,NaN,1.0,21.0,0.0,2023-01-01,3040.0,144.76
3,382488,spotahome,Camden,studios,Studio,1755,"Studio flat to rent in Camden Town, London",furnished,1755,NaN,1.0,NaN,0.0,2023-01-11,1755.0,NaN
4,619142,spotahome,Lambeth,apartments/bedrooms:1,Apartment,4500,"One bedroom apartment for rent in Brixton, London",furnished,4500,NaN,1.0,NaN,1.0,2022-10-04,4500.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831,643947,spotahome,Islington,apartments/bedrooms:3,Apartment,7350,"3-bedroom apartment for rent in Islington, London",furnished,7350,NaN,3.0,NaN,3.0,2022-11-07,7350.0,NaN
832,641245,spotahome,Hackney,apartments/bedrooms:3,Apartment,6000,"3-bedroom apartment for rent in Haggerston, Lo...",furnished,6000,NaN,NaN,NaN,3.0,2022-11-26,6000.0,NaN
833,506930,spotahome,Hackney,apartments/bedrooms:3,Apartment,4941,"3-Bedroom Apartment for rent in Old Street, Lo...",furnished,4941,NaN,2.0,90.0,3.0,2023-01-14,4941.0,54.90
834,843222,spotahome,Hackney,apartments/bedrooms:3,Apartment,8250-10500,3-bedroom apartment for rent in London,furnished,8250,10500.0,1.0,NaN,3.0,2023-01-08,9375.0,NaN


In [17]:
spotahome_eda.eval('price_per_bedroom = avg_price_sqm / bedrooms', inplace=True)

In [18]:
spotahome_eda

,platform_id,platform,neighborhood,property_type,housing_type,price_pcm,title,furniture,price_pcm_0,price_pcm_1,bathrooms,size_sqm,bedrooms,available_from,avg_price_sqm,price_per_sqm,price_per_bedroom
0,654316,spotahome,Kensington and Chelsea,apartments/bedrooms:1,Apartment,3690,Stunning Chelsea Nest,furnished,3690,NaN,1.0,NaN,1.0,2023-04-02,3690.0,NaN,3690.0
1,726546,spotahome,Lambeth,apartments/bedrooms:2,Apartment,3924-8856,2-bedroom apartment for rent in Patmore Estate...,furnished,3924,8856.0,2.0,72.0,2.0,2022-11-06,6390.0,88.75,3195.0
2,438447,spotahome,Kensington and Chelsea,studios,Studio,3040,Studio in London,furnished,3040,NaN,1.0,21.0,0.0,2023-01-01,3040.0,144.76,inf
3,382488,spotahome,Camden,studios,Studio,1755,"Studio flat to rent in Camden Town, London",furnished,1755,NaN,1.0,NaN,0.0,2023-01-11,1755.0,NaN,inf
4,619142,spotahome,Lambeth,apartments/bedrooms:1,Apartment,4500,"One bedroom apartment for rent in Brixton, London",furnished,4500,NaN,1.0,NaN,1.0,2022-10-04,4500.0,NaN,4500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831,643947,spotahome,Islington,apartments/bedrooms:3,Apartment,7350,"3-bedroom apartment for rent in Islington, London",furnished,7350,NaN,3.0,NaN,3.0,2022-11-07,7350.0,NaN,2450.0
832,641245,spotahome,Hackney,apartments/bedrooms:3,Apartment,6000,"3-bedroom apartment for rent in Haggerston, Lo...",furnished,6000,NaN,NaN,NaN,3.0,2022-11-26,6000.0,NaN,2000.0
833,506930,spotahome,Hackney,apartments/bedrooms:3,Apartment,4941,"3-Bedroom Apartment for rent in Old Street, Lo...",furnished,4941,NaN,2.0,90.0,3.0,2023-01-14,4941.0,54.90,1647.0
834,843222,spotahome,Hackney,apartments/bedrooms:3,Apartment,8250-10500,3-bedroom apartment for rent in London,furnished,8250,10500.0,1.0,NaN,3.0,2023-01-08,9375.0,NaN,3125.0


In [ ]:
spotahome_eda.describe()

In [ ]:
spotahome_eda['neighborhood'].value_counts()

In [ ]:
spotahome_eda['neighborhood'].value_counts().plot(kind='bar');

In [ ]:
spotahome_eda.groupby('neighborhood')['price_per_sqm'].mean().sort_values().plot(kind='bar');

In [ ]:
# call the schema created for this project
schema = 'capstone_jmrs'
# get the function to connect to the database
engine = get_engine()

# give the table a unique name
table_name = 'spotahome_eda'

# import the table to sql
if engine != None:
    try:
        spotahome_eda.to_sql(name=table_name,
                                   con=engine,
                                   if_exists='replace',
                                   schema=schema,
                                   index=False,
                                   chunksize=5000,
                                   method='multi')
        print(f"The {table_name} table was imported successfully.")

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None
